<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/Sector_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-02-04 11:44:48


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-10*365)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
# fetch nse indices scrips

import pandas as pd
import numpy as np

df_nse_sectors = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/NSE_Sectors_Indices.csv')

In [6]:
sectors = df_nse_sectors['Sector'].unique()
df_sectors = pd.DataFrame()
for sector in sectors:
  stocks = df_nse_sectors[df_nse_sectors['Sector'] == sector]['Symbol'].values
  df_sector = pd.DataFrame()
  tweight = 0
  for stock in stocks:
    stock_df = yfin.Ticker(stock+'.NS').history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    col = ['Close']
    stock_df = stock_df[col]
    stock_df['Split'] = 1
    weight = df_nse_sectors[df_nse_sectors['Symbol'] == stock]['Weight'].values[0]
    tweight = sum(df_nse_sectors[df_nse_sectors['Sector'] == sector]['Weight'].values)
    if stock == 'NESTLEIND':
      stock_df.loc[stock_df.index < '2024-01-01', 'Split'] = 10
    #stock_df[stock] = (stock_df[col]*weight)/(tweight*stock_df['Split'].values[0])
    stock_df[stock] = stock_df['Close']/stock_df['Split']
    col = [stock]
    stock_df = stock_df[col]
    df_sector = pd.concat([df_sector, stock_df], axis=1)

  df_sector[sector] = df_sector[stocks].sum(axis=1)
  #col = [sector]
  #df_sector = df_sector[col]
  df_sectors = pd.concat([df_sectors, df_sector], axis=1)
df_sectors

,TATAMOTORS,M&M,MARUTI,BAJAJ-AUTO,HEROMOTOCO,TVSMOTOR,AUTO,HDFCBANK,ICICIBANK,SBIN,...,NETWORK18,NAZARA,MEDIA,TATASTEEL,HINDALCO,ADANIENT,JSWSTEEL,VEDL,JINDALSTEL,METAL
Date,,,,,,,,,,,,,,,,,,,,,
2019-02-04 00:00:00+05:30,179.689957,641.602234,6680.092285,2166.735596,2374.923340,485.874115,12528.917526,1015.539795,347.687683,271.748322,...,35.599998,NaN,847.408451,40.639473,197.280258,121.641953,240.683075,79.125992,129.553070,808.923820
2019-02-05 00:00:00+05:30,175.253784,651.543274,6780.845215,2222.615723,2438.479004,483.637482,12752.374481,1019.954407,345.824493,272.418213,...,35.049999,NaN,861.343084,39.670544,197.811600,122.778328,240.727020,79.738976,127.820396,808.546864
2019-02-06 00:00:00+05:30,177.945404,661.246460,6866.685059,2271.403564,2460.262695,475.031555,12912.574738,1024.103638,352.345795,276.629150,...,34.750000,NaN,901.805389,41.412048,204.236267,121.444321,245.787094,80.302925,133.810425,826.993080
2019-02-07 00:00:00+05:30,182.281891,666.764099,6980.582031,2339.828857,2510.662842,490.833527,13170.953247,1021.498230,351.708374,275.050049,...,35.750000,NaN,913.758015,41.749256,202.110809,122.481888,240.331024,80.155807,134.949036,821.777821
2019-02-08 00:00:00+05:30,150.231766,648.879578,6819.702148,2338.025146,2510.492188,479.310181,12946.641006,1024.103638,348.227112,273.710205,...,34.349998,NaN,892.739536,40.016281,198.922638,122.284264,231.310928,75.595085,128.760986,796.890182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-29 00:00:00+05:30,841.000000,1640.000000,9991.400391,7548.799805,4593.500000,1964.699951,26579.400146,1454.650024,1016.400024,622.950012,...,117.000000,893.400024,3389.300026,135.000000,567.900024,3064.649902,811.700012,267.799988,730.750000,5577.799927
2024-01-30 00:00:00+05:30,858.849976,1620.349976,9951.650391,7566.399902,4588.350098,1957.599976,26543.200317,1444.300049,1016.900024,626.750000,...,115.550003,870.000000,3337.000046,134.699997,569.250000,3091.100098,812.000000,266.250000,746.200012,5619.500107
2024-01-31 00:00:00+05:30,884.200012,1651.550049,10186.900391,7667.549805,4621.549805,2001.349976,27013.100037,1462.550049,1028.150024,640.500000,...,117.000000,889.349976,3355.899956,135.949997,579.349976,3142.000000,818.650024,273.850006,757.150024,5706.950027


In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = df_sectors[[stock_symbol]]
    stock_df.columns = ['Close']

    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Sector'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
    #stock_df['Sector'] = stock_symbol
    stock_df = stock_df.tail(1)
    return stock_df

In [8]:
df_final =  pd.DataFrame()
for sector in sectors:

  df_tmp = stock_prec_dev(sector)
  df_final = df_final.append(df_tmp)
# all sectors
df_final = df_final.sort_values(by = 'ATH%', ascending=False)
data_table.DataTable(df_final, include_index=False)

,Close,200_SMA,Dev%_200,Sector,Max,Min,RSI_14,ATH%
Date,,,,,,,,
2024-02-02 00:00:00+05:30,3295.00,3129.0,5.31,MEDIA,3808.40,1273.44,49.0,15.58
2024-02-02 00:00:00+05:30,14550.50,12822.0,13.48,IT,16403.73,10640.54,53.0,12.74
2024-02-02 00:00:00+05:30,5751.75,4734.0,21.50,METAL,6262.29,3173.78,66.0,8.88
2024-02-02 00:00:00+05:30,9870.25,9474.0,4.18,FINANCE,10627.62,6955.03,44.0,7.67
2024-02-02 00:00:00+05:30,6009.80,5970.0,0.67,BANKS,6391.75,5339.05,44.0,6.36
2024-02-02 00:00:00+05:30,12971.45,11904.0,8.97,FMCG,13527.44,10048.88,50.0,4.29
2024-02-02 00:00:00+05:30,17643.35,15091.0,16.91,PHARMA,17748.05,11486.18,57.0,0.59
2024-02-02 00:00:00+05:30,27695.75,21972.0,26.05,AUTO,27695.75,16473.19,78.0,0.00
